
# REST em Python: Princípios, HTTP e APIs com Flask & FastAPI

**Objetivo:** Este notebook apresenta, de forma prática, os princípios do **REST**, os **métodos HTTP** e **status codes**, além de exemplos de criação de **APIs RESTful** em Python com **Flask** e **FastAPI**.  
> Observação: os exemplos usam Python. *Express.js* é citado apenas para contexto (Node.js), sem código aqui.



## Sumário
1. [Princípios REST e boas práticas](#principios)
2. [Métodos HTTP e status codes](#http)
3. [API RESTful com Flask (passo a passo)](#flask)
4. [API RESTful com FastAPI (passo a passo)](#fastapi)
5. [Testes de API (requests/httpx)](#testes)
6. [Segurança: HTTPS, CORS, Auth (JWT)](#seguranca)
7. [Boas práticas extras e checklist](#boas-praticas)



## 1. Princípios REST e boas práticas <a id="principios"></a>

REST (Representational State Transfer) é um estilo arquitetural para sistemas distribuídos. Principais ideias:

- **Recursos e URIs claras**: modele o domínio como **recursos** (substantivos) e use URIs previsíveis: `/users`, `/users/{id}`, `/orders/{id}/items`.
- **Métodos HTTP semânticos**: `GET` (ler), `POST` (criar), `PUT` (substituir), `PATCH` (atualizar parcialmente), `DELETE` (remover).
- **Stateless**: o servidor **não mantém estado de sessão** entre requisições; cada request carrega o contexto necessário (ex.: token de auth). Possibilita o uso de load-balancers. Mais simples de tratar. Como não há sincronização, fica mais fácil de fazer cache dos resultados.
- **Cacheable**: use cabeçalhos de cache (`ETag`, `Cache-Control`, `Last-Modified`) quando possível.
- **Uniform interface**: formatos consistentes (ex.: JSON), hipermídia opcional ([HATEOAS](https://en.wikipedia.org/wiki/HATEOAS)).
- **Camadas (layered system)**: gateways, API gateways, proxies, balanceadores.
- **Código sob demanda (opcional)**: o servidor pode enviar códigos executáveis para o cliente.
- **Versionamento**: `/v1/...` na URI ou `Accept: application/vnd.suaapi.v1+json`.
- **Paginação/Filtragem/Ordenação**: `?page=1&size=20`, `?sort=-created_at`, `?status=OPEN`.
- **Contrato e documentação**: **OpenAPI/Swagger** para descrever endpoints, schemas e respostas.
- **Erros padronizados**: inclua `type`, `title`, `detail`, `instance` (RFC 7807) ou um schema consistente.
- **Segurança**: **HTTPS** obrigatório, autenticação (ex.: **JWT**/OAuth2), autorização por escopo/role, **CORS** configurado.
- **Observabilidade**: logs estruturados (correlation id), métricas, tracing.
- **Idempotência**: `GET`, `PUT`, `DELETE`, `HEAD`, `OPTIONS` devem ser idempotentes; use **Idempotency-Key** em criações sensíveis.



## 2. Métodos HTTP e status codes <a id="http"></a>

### Métodos (semântica e idempotência)
| Método   | Semântica                         | Seguro | Idempotente |
|---------|-----------------------------------|:------:|:-----------:|
| GET     | Lê recurso(s)                     |  ✅   |     ✅      |
| HEAD    | Cabeçalhos de um GET              |  ✅   |     ✅      |
| OPTIONS | Capacidades / CORS preflight      |  ✅   |     ✅      |
| POST    | Cria/aciona processamento         |  ❌ (muda servidor)  |     ❌      |
| PUT     | Substitui recurso por inteiro     |  ❌ (muda recurso)  |     ✅      |
| PATCH   | Atualiza parcialmente             |  ❌ (muda recurso)  |     ❌*     |
| DELETE  | Remove recurso                    |  ❌ (muda recurso)   |     ✅      |

\* PATCH pode ser implementado de forma idempotente, mas **não é garantido**.

### Status codes úteis
- **2xx Sucesso**: `200 OK`, `201 Created` (com `Location` do novo recurso), `204 No Content` (sem corpo).
- **3xx Redirecionamento**: `304 Not Modified` (cache condicional).
- **4xx Erro do cliente**: `400 Bad Request`, `401 Unauthorized`, `403 Forbidden`, `404 Not Found`, `409 Conflict` (ex.: duplicidade), `422 Unprocessable Entity` (validação).
- **5xx Erro do servidor**: `500 Internal Server Error`, `503 Service Unavailable`.


In [1]:

# (Opcional) Instalar dependências no ambiente local do usuário:
# Execute esta célula localmente, se necessário.
!pip install flask fastapi "uvicorn[standard]" pydantic httpx python-multipart


  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.1 MB/s eta 0:00:00a 0:00:01
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.4 MB/s eta 0:00:00a 0:00:01
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)

[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip



## 3. API RESTful com FastAPI <a id="fastapi"></a>

**FastAPI** oferece tipagem moderna, validação com **Pydantic**, e **OpenAPI/Swagger** automático em `/docs` e `/redoc`.

Destaques:
- Suporte a **async**/**await** e alta performance (Uvicorn + Starlette).
- Validação e documentação automáticas a partir de **type hints**.
- Dependências e injeção de *dependencies* (ex.: autenticação).

> Rodar localmente: `uvicorn app_fastapi:app --reload`  
> Docs: `http://127.0.0.1:8000/docs`


In [ ]:

# app_fastapi.py
from typing import List, Optional, Dict
from fastapi import FastAPI, HTTPException, Query, Depends, Header, status
from pydantic import BaseModel, Field

app = FastAPI(title="Todos API", version="1.0.0")

class TodoIn(BaseModel):
    title: str = Field(min_length=1, max_length=140)
    done: bool = False

class Todo(TodoIn):
    id: int

DB: Dict[int, Todo] = {}
_next_id = 1

def _new_id():
    global _next_id
    i = _next_id
    _next_id += 1
    return i

# Exemplo simples de auth por Bearer Token (não use em produção sem JWT/OAuth2!)
def get_token(authorization: Optional[str] = Header(default=None)):
    if not authorization or not authorization.startswith("Bearer "):
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Missing or invalid token")
    token = authorization.split(" ", 1)[1]
    if token != "secrettoken":  # placeholder
        raise HTTPException(status_code=status.HTTP_403_FORBIDDEN, detail="Forbidden")
    return token

@app.get("/todos", response_model=Dict[str, object])
async def list_todos(
    page: int = Query(1, ge=1),
    size: int = Query(10, ge=1, le=100),
    token: str = Depends(get_token)  # exemplo de dependência
):
    items = list(DB.values())
    start = (page - 1) * size
    end = start + size
    data = [t.model_dump() for t in items[start:end]]
    return {"data": data, "page": page, "size": size, "total": len(items)}

@app.post("/todos", status_code=status.HTTP_201_CREATED, response_model=Todo)
async def create_todo(todo_in: TodoIn, token: str = Depends(get_token)):
    new_id = _new_id()
    todo = Todo(id=new_id, **todo_in.model_dump())
    DB[new_id] = todo
    return todo

@app.get("/todos/{todo_id}", response_model=Todo)
async def get_todo(todo_id: int, token: str = Depends(get_token)):
    todo = DB.get(todo_id)
    if not todo:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Not found")
    return todo

@app.put("/todos/{todo_id}", response_model=Todo)
async def replace_todo(todo_id: int, todo_in: TodoIn, token: str = Depends(get_token)):
    if todo_id not in DB:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Not found")
    todo = Todo(id=todo_id, **todo_in.model_dump())
    DB[todo_id] = todo
    return todo

@app.patch("/todos/{todo_id}", response_model=Todo)
async def update_todo(todo_id: int, todo_in: TodoIn, token: str = Depends(get_token)):
    todo = DB.get(todo_id)
    if not todo:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Not found")
    data = todo.model_dump()
    patch = todo_in.model_dump(exclude_unset=True)
    data.update(patch)
    updated = Todo(**data)
    DB[todo_id] = updated
    return updated

@app.delete("/todos/{todo_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_todo(todo_id: int, token: str = Depends(get_token)):
    if todo_id not in DB:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Not found")
    del DB[todo_id]
    return None


## 4. Testando a API no Insomnia

🔹 1. Instalar o Insomnia

Baixe em: https://insomnia.rest/download

🔹 2. Explicação prática.


## 5. Segurança: HTTPS, CORS, Auth (JWT) <a id="seguranca"></a>

- **HTTPS** sempre (TLS), principalmente em produção.
- **CORS**: habilite apenas origens necessárias (evite `*` em produção). Por padrão, o CORS somente autoriza requisições pro/do mesmo domínio.
- **Autenticação/Autorização**: JWT/OAuth2 (escopos por recurso/ação).
- **Rate limiting** e *throttling* para mitigar abuso.
- **Headers de segurança**: `Strict-Transport-Security`, `X-Content-Type-Options`, `Content-Security-Policy` (para front-ends).
- **Proteja segredos** com variáveis de ambiente e *secret managers*.

**Exemplo CORS (FastAPI)**:


In [ ]:

# cors_fastapi_snippet.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["https://seu-frontend.com"],
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "PATCH", "DELETE", "OPTIONS"],
    allow_headers=["*"],
)


## 6. Autenticando a API com JWT

### O que é JWT
**JWT (JSON Web Token)** é um padrão para representar **tokens de acesso** de forma compacta e segura.  
Usado principalmente para **autenticação** e **autorização** em APIs.

Um token JWT é formado por três partes (separadas por ponto):

`header.payload.signature``

- **Header** → algoritmo de assinatura (`HS256`, `RS256`)  
- **Payload** → dados (*claims*) como `sub` (usuário), `exp` (expiração), `scope` (permissões)  
- **Signature** → garante integridade do token  

### Fluxo de autenticação
1. **Login**
   - Cliente envia `username` e `password`.  
   - Servidor valida e retorna:
     - `access_token` (curta duração, ex. 15 min)  
     - `refresh_token` (longa duração, ex. 7 dias).  

2. **Acesso a rotas protegidas**
   - Cliente envia o header:
     ```
     Authorization: Bearer <ACCESS_TOKEN>
     ```
   - Servidor valida o token antes de responder.  

3. **Renovação**
   - Quando o `access_token` expira, cliente usa o `refresh_token` para obter um novo.  


## 7. Boas práticas extras e checklist <a id="boas-praticas"></a>

**Checklist resumido:**
- [ ] URIs e recursos bem modelados (substantivos, plural consistente).
- [ ] Versionamento claro (URI ou media type).
- [ ] Uso correto de métodos HTTP, idempotência e status codes.
- [ ] Schemas de request/response com validação (Pydantic), *error shape* padronizado.
- [ ] Paginação, filtragem e ordenação consistentes.
- [ ] Observabilidade: logs estruturados, métricas, tracing.
- [ ] Segurança: HTTPS, JWT/OAuth2, CORS restrito, rate limiting.
- [ ] Performance: gzip/br, caching, ETags, *timeouts*, *connection pooling*.

> Dica: para um projeto maior, use **routers** (caminhos), *service layer* (lógica de negócio), *repository pattern* (dados separados da lógica de negócios), e **pydantic settings** para configuração.
